In [1]:
import pickle
import os
import numpy as np
from keras.layers import LSTM, Input, Dense, Activation, Embedding, Concatenate, Reshape
from keras.layers import RepeatVector, Permute
from keras.layers import Multiply, Lambda
import keras.backend as K 
from keras.models import Model
import tensorflow as tf

2023-05-22 17:40:16.932540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 17:40:20.322548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-22 17:40:20.322712: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pickle.load(input_file)


durations_to_int = dictionary["durations_to_int"]
int_to_duration = dictionary["int_to_duration"]
notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]
bars_to_int = dictionary["bars_to_int"]
int_to_bars = dictionary["int_to_bars"]

n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)
n_bars = len(bars_to_int)

In [3]:
def create_network(n_notes, n_durations, n_bars, n_phrases, embed_size = 100, rnn_units = 256, use_attention = False):

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))
    bars_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in)
    x3 = Embedding(n_bars, embed_size)(bars_in)

    x = Concatenate()([x1,x2,x3])

    x = LSTM(rnn_units, return_sequences=True)(x)

    if use_attention:

        x = LSTM(rnn_units, return_sequences=True)(x)

        e = Dense(1, activation='tanh')(x)
        e = Reshape([-1])(e)
        alpha = Activation('softmax')(e)

        alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

        c = Multiply()([x, alpha_repeated])
        c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
    
    else:
        c = LSTM(rnn_units)(x)
                                    
    phrases_out = Dense(n_phrases, activation = 'softmax', name = 'phrase')(c)
   
    model = Model([notes_in, durations_in, bars_in], [phrases_out])
    

    if use_attention:
        att_model = Model([notes_in, durations_in, bars_in], alpha)
    else:
        att_model = None


    opti = tf.keras.optimizers.Adam(learning_rate=0.001) #RMSprop(lr = 0.001)
    model.compile(loss=['binary_crossentropy', 'binary_crossentropy'], optimizer=opti)

    return model, att_model

In [4]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 46
rnn_units = 128
use_attention = True

model, att_model = create_network(n_notes, n_durations, n_bars, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

2023-05-22 17:40:23.957052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-22 17:40:23.957098: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-22 17:40:23.957127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (umn-1684688989): /proc/driver/nvidia/version does not exist
2023-05-22 17:40:23.957405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the a

ValueError: Cannot assign value to variable ' lstm/lstm_cell/kernel:0': Shape mismatch.The variable shape (138, 512), and the assigned value shape (138, 2048) are incompatible.

In [ ]:
pieces_train = None
with open('pieces_train.pkl', 'rb') as handle:
    pieces_train = pickle.load(handle)

pieces_test = None
with open('pieces_test.pkl', 'rb') as handle:
    pieces_test = pickle.load(handle)

In [ ]:
def get_boundaries(piece):
    notes_input = [ notes_to_int[note] for note in piece[0] ]
    durations_input = [ durations_to_int[duration] for duration in piece[1] ]
    bars_input = [ bars_to_int[bar] for bar in piece[2] ]
    expected = phrases_input = piece[3]
    
    max_extra_notes = len(notes_input) - 1
    seq_len = 5
    phrases_output = []
    phrases_output.append(0) # after the first note (that is already in the initial sequence)

    notes_input_sequence = [notes_input[0]] # the first note element of the sequence
    durations_input_sequence = [durations_input[0]] # the first duration element of the sequence
    bars_input_sequence = [bars_input[0]] # the first duration element of the sequence

    if seq_len is not None:
        notes_input_sequence = [notes_to_int[129]] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
        durations_input_sequence = [0] * (seq_len - len(durations_input_sequence)) + durations_input_sequence # the same for durations
        bars_input_sequence =  [0] * (seq_len - len(bars_input_sequence)) + bars_input_sequence # the same for durations

    for note_index in range(max_extra_notes):
        prediction_input = [
            np.array([notes_input_sequence]),
            np.array([durations_input_sequence]),
            np.array([bars_input_sequence])
        ]

        phrases_prediction = model.predict(prediction_input, verbose=0)
        phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

        notes_input_sequence.append(notes_input[note_index])
        durations_input_sequence.append(durations_input[note_index])
        bars_input_sequence.append(bars_input[note_index])
        
        if len(notes_input_sequence) > seq_len:
            notes_input_sequence = notes_input_sequence[1:]
            durations_input_sequence = durations_input_sequence[1:]
            bars_input_sequence = bars_input_sequence[1:]
    actual = phrases_output
    
    return (actual, expected)

### Calculate F1 score

In [ ]:
def f1_score(expected, actual):
    """
    Calculates the F1 score given two arrays of 0s and 1s (expected and actual).
    """
    # Ensure that the inputs are numpy arrays.
    expected = np.array(expected)
    actual = np.array(actual)
    
    # Calculate the true positives, false positives, and false negatives.
    tp = np.sum((expected == 1) & (actual == 1))
    fp = np.sum((expected == 0) & (actual == 1))
    fn = np.sum((expected == 1) & (actual == 0))
    
    # Calculate the precision, recall, and F1 score.
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    
    return f1

F1 for the train data

In [ ]:
actual = []
expected = []
for piece in pieces_train:
    a, e = get_boundaries(piece)
    actual.extend(a)
    expected.extend(e)
f1_score(expected,actual)

F1 for the test data

In [ ]:
actual = []
expected = []
for piece in pieces_test:
    a, e = get_boundaries(piece)
    actual.extend(a)
    expected.extend(e)
f1_score(expected,actual)